In [1]:
# Essentials
import numpy as np
import pandas as pd
import datetime
import random
import pickle
# Plots
import matplotlib.pyplot as plt

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor 

# Misc
from sklearn.model_selection import KFold, cross_val_score
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

In [2]:
cd ./Data

c:\Users\Tan\OneDrive - Hanoi University of Science and Technology\Documents\ĐATN-20220819T101500Z-001\ĐATN\Data


C:\Users\Tan\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
bottle = pd.read_csv('bottle.csv') 
cast = pd.read_csv('cast.csv')


C:\Users\Tan\AppData\Local\Temp\ipykernel_20176\3589876794.py:1: DtypeWarning: Columns (47,73) have mixed types. Specify dtype option on import or set low_memory=False.
  bottle = pd.read_csv('bottle.csv')
C:\Users\Tan\AppData\Local\Temp\ipykernel_20176\3589876794.py:2: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  cast = pd.read_csv('cast.csv')


In [5]:
bottle.shape

(864863, 74)

In [6]:
bottle.head()

,Cst_Cnt,Btl_Cnt,Sta_ID,Depth_ID,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,...,R_PHAEO,R_PRES,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1,DIC Quality Comment
0,1,1,054.0 056.0,19-4903CR-HY-060-0930-05400560-0000A-3,0,10.50,33.440,NaN,25.649,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,054.0 056.0,19-4903CR-HY-060-0930-05400560-0008A-3,8,10.46,33.440,NaN,25.656,NaN,...,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,054.0 056.0,19-4903CR-HY-060-0930-05400560-0010A-7,10,10.46,33.437,NaN,25.654,NaN,...,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,054.0 056.0,19-4903CR-HY-060-0930-05400560-0019A-3,19,10.45,33.420,NaN,25.643,NaN,...,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,054.0 056.0,19-4903CR-HY-060-0930-05400560-0020A-7,20,10.45,33.421,NaN,25.643,NaN,...,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
bottle.drop(columns= ['Depth_ID','Sta_ID'],axis = 0, inplace =True)

In [8]:
nul = bottle.isnull().sum() / len(bottle)
nul = nul.sort_values(ascending = False)
nul.tail(36)

PO4uM          0.522101
R_PO4          0.522092
PO4q           0.477621
SiO3qu         0.409310
NO2q           0.387794
NO3q           0.387264
Chlqua         0.260963
Phaqua         0.260958
Oxy_µmol/Kg    0.235407
O2Sat          0.235400
R_O2Sat        0.229418
P_qual         0.220969
O2ml_L         0.195016
R_O2           0.195016
NH3q           0.065402
R_SIGMA        0.061115
R_SVA          0.061017
STheta         0.060922
Salnty         0.054753
R_SALINITY     0.054753
S_prec         0.054753
R_DYNHT        0.053947
R_POTEMP       0.053242
MeanAq         0.028240
DarkAq         0.028239
C14A1q         0.018798
C14A2q         0.018778
R_TEMP         0.012676
T_degC         0.012676
T_prec         0.012676
Depthm         0.000000
RecInd         0.000000
R_PRES         0.000000
R_Depth        0.000000
Btl_Cnt        0.000000
Cst_Cnt        0.000000
dtype: float64

In [9]:
categorical = [feature for feature in bottle.columns if bottle[feature].nunique() < 15]
categorical

['RecInd',
 'T_prec',
 'T_qual',
 'S_prec',
 'S_qual',
 'P_qual',
 'O_qual',
 'SThtaq',
 'O2Satq',
 'Chlqua',
 'Phaqua',
 'PO4q',
 'SiO3qu',
 'NO2q',
 'NO3q',
 'NH3q',
 'C14A1p',
 'C14A1q',
 'C14A2p',
 'C14A2q',
 'DarkAp',
 'DarkAq',
 'MeanAp',
 'MeanAq',
 'pH2']

In [10]:
bottle.nunique()

Cst_Cnt                 34404
Btl_Cnt                864863
Depthm                   3219
T_degC                  13847
Salnty                  14602
                        ...  
TA1                      1849
TA2                       225
pH2                        10
pH1                        82
DIC Quality Comment        37
Length: 72, dtype: int64

In [11]:
col = nul.tail(36).index.values
col = col

In [12]:
bottle = bottle[col]
bottle.shape

(864863, 36)

In [13]:
bottle.describe()

,PO4uM,R_PO4,PO4q,SiO3qu,NO2q,NO3q,Chlqua,Phaqua,Oxy_µmol/Kg,O2Sat,...,C14A2q,R_TEMP,T_degC,T_prec,Depthm,RecInd,R_PRES,R_Depth,Btl_Cnt,Cst_Cnt
count,413317.000000,413325.000000,451786.000000,510866.000000,529474.000000,529933.000000,639166.000000,639170.000000,661268.000000,661274.000000,...,848623.000000,853900.000000,853900.000000,853900.000000,864863.000000,864863.000000,864863.000000,864863.000000,864863.000000,864863.000000
mean,1.564747,1.564736,8.999320,8.999687,8.995458,8.993122,8.999848,8.999844,148.808694,57.103779,...,8.999992,10.799697,10.799677,2.017148,226.831951,4.700273,228.395694,226.832495,432432.000000,17138.790958
std,1.035966,1.035973,0.026895,0.018768,0.077426,0.131516,0.012318,0.012507,90.187533,37.094137,...,0.002872,4.243829,4.243825,0.148498,316.050259,1.877428,319.456731,316.050007,249664.587269,10240.949817
min,0.000000,0.000000,4.000000,4.000000,4.000000,4.000000,8.000000,8.000000,-0.434900,-0.100000,...,8.000000,1.440000,1.440000,1.000000,0.000000,3.000000,0.000000,0.000000,1.000000,1.000000
25%,0.470000,0.470000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,60.915470,21.100000,...,9.000000,7.680000,7.680000,2.000000,46.000000,3.000000,46.000000,46.000000,216216.500000,8269.000000
50%,1.570000,1.570000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,151.064150,54.400000,...,9.000000,10.060000,10.060000,2.000000,125.000000,3.000000,126.000000,125.000000,432432.000000,16848.000000
75%,2.480000,2.480000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,240.379600,97.600000,...,9.000000,13.880000,13.880000,2.000000,300.000000,7.000000,302.000000,300.000000,648647.500000,26557.000000
max,5.210000,5.210000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,485.701800,214.100000,...,9.000000,31.140000,31.140000,3.000000,5351.000000,7.000000,5458.000000,5351.000000,864863.000000,34404.000000


In [14]:
bottle.to_csv("new_bottle.csv")

## Cast 

In [15]:
cast.head()

,Cst_Cnt,Cruise_ID,Cruise,Cruz_Sta,DbSta_ID,Cast_ID,Sta_ID,Quarter,Sta_Code,Distance,...,Wave_Prd,Wind_Dir,Wind_Spd,Barometer,Dry_T,Wet_T,Wea,Cloud_Typ,Cloud_Amt,Visibility
0,1,1949-03-01-C-31CR,194903,19490305400560,5400560,19-4903CR-HY-060-0930-05400560,054.0 056.0,1,NST,NaN,...,NaN,23.0,18.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
1,2,1949-03-01-C-31CR,194903,19490305200750,5200750,19-4903CR-HY-060-2112-05200750,052.0 075.0,1,NST,NaN,...,NaN,16.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
2,3,1949-03-01-C-31CR,194903,19490305100850,5100850,19-4903CR-HY-061-0354-05100850,051.0 085.0,1,NST,NaN,...,NaN,23.0,5.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN
3,4,1949-03-01-C-31CR,194903,19490305000950,5000950,19-4903CR-HY-061-1042-05000950,050.0 095.0,1,NST,NaN,...,NaN,18.0,8.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,5,1949-03-01-C-31CR,194903,19490305001040,5001040,19-4903CR-HY-061-1706-05001040,050.0 104.0,1,NST,NaN,...,NaN,27.0,13.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN


In [16]:
cast.shape

(34404, 61)

In [17]:
nul_cast = cast.isnull().sum() / len(cast)
nul_cast = nul_cast.sort_values(ascending = False)
nul_cast.tail(50)

Order_Occ      0.710673
Wave_Prd       0.661842
Cloud_Amt      0.654633
Wave_Ht        0.648355
IntChl         0.645303
Wave_Dir       0.634984
Data_Or        0.468812
Wet_T          0.448058
Barometer      0.445907
Dry_T          0.445675
Distance       0.426985
Wea            0.220817
Bottom_D       0.066300
Ship_Name      0.049122
Wind_Spd       0.048802
Wind_Dir       0.042902
Time           0.009418
Event_Num      0.000087
DbSta_ID       0.000000
Cast_ID        0.000000
Sta_ID         0.000000
Quarter        0.000000
Cruz_Sta       0.000000
Lat_Dec        0.000000
Cruise         0.000000
Sta_Code       0.000000
Date           0.000000
Year           0.000000
Month          0.000000
Julian_Date    0.000000
Julian_Day     0.000000
Rpt_Line       0.000000
Lat_Min        0.000000
Lat_Deg        0.000000
Lon_Min        0.000000
Ac_Line        0.000000
Rpt_Sta        0.000000
St_Station     0.000000
Ac_Sta         0.000000
Cruise_ID      0.000000
Lon_Hem        0.000000
Ship_Code      0

In [18]:
col = ['Cst_Cnt','Distance','Wind_Spd','Wave_Ht','Wave_Prd','IntChl','Dry_T']
col

['Cst_Cnt', 'Distance', 'Wind_Spd', 'Wave_Ht', 'Wave_Prd', 'IntChl', 'Dry_T']

In [19]:
cast = cast[col]
cast.isnull().sum()

Cst_Cnt         0
Distance    14690
Wind_Spd     1679
Wave_Ht     22306
Wave_Prd    22770
IntChl      22201
Dry_T       15333
dtype: int64

In [20]:
cast.nunique()

Cst_Cnt     34404
Distance     9350
Wind_Spd       49
Wave_Ht        25
Wave_Prd       28
IntChl       1710
Dry_T         220
dtype: int64

In [21]:
cast.dtypes

Cst_Cnt       int64
Distance    float64
Wind_Spd    float64
Wave_Ht     float64
Wave_Prd    float64
IntChl      float64
Dry_T       float64
dtype: object

In [22]:
cast.isnull()

,Cst_Cnt,Distance,Wind_Spd,Wave_Ht,Wave_Prd,IntChl,Dry_T
0,False,True,False,True,True,True,True
1,False,True,False,True,True,True,True
2,False,True,False,True,True,True,True
3,False,True,False,True,True,True,True
4,False,True,False,True,True,True,True
...,...,...,...,...,...,...,...
34399,False,False,False,True,True,False,False
34400,False,False,False,True,True,False,False
34401,False,False,False,False,False,False,False
34402,False,False,False,False,False,False,False


In [23]:
cast.to_csv("new_cast.csv")

In [ ]:
# new_bottle = pd.read_csv('new_bottle.csv') 
# new_cast = pd.read_csv('new_cast.csv')

## Join Data

In [ ]:
data = bottle.set_index('Cst_Cnt').join(cast.set_index('Cst_Cnt'))
data.shape

(864863, 41)

In [27]:
categorical = [feature for feature in data.columns if data[feature].nunique() < 15]
categorical

['PO4q',
 'SiO3qu',
 'NO2q',
 'NO3q',
 'Chlqua',
 'Phaqua',
 'P_qual',
 'NH3q',
 'S_prec',
 'MeanAq',
 'DarkAq',
 'C14A1q',
 'C14A2q',
 'T_prec',
 'RecInd']

In [28]:
data.drop(columns = categorical, axis= 0, inplace = True)

In [29]:
data.nunique()

PO4uM             451
R_PO4             451
Oxy_µmol/Kg    391814
O2Sat           10379
R_O2Sat          1453
O2ml_L           6500
R_O2              871
R_SIGMA          4398
R_SVA            5254
STheta          36368
Salnty          14602
R_SALINITY       3584
R_DYNHT          1848
R_POTEMP         2857
R_TEMP           2819
T_degC          13847
Depthm           3219
R_PRES           3328
R_Depth          3219
Btl_Cnt        864863
Distance         9350
Wind_Spd           49
Wave_Ht            25
Wave_Prd           28
IntChl           1710
Dry_T             220
dtype: int64

In [30]:
data.drop(columns = ['PO4uM','Depthm','R_TEMP','R_O2Sat','R_O2','Oxy_µmol/Kg','R_SALINITY','R_POTEMP','R_PRES','R_SVA','R_SIGMA'], axis =0,inplace =True)

In [31]:
data.nunique()

R_PO4          451
O2Sat        10379
O2ml_L        6500
STheta       36368
Salnty       14602
R_DYNHT       1848
T_degC       13847
R_Depth       3219
Btl_Cnt     864863
Distance      9350
Wind_Spd        49
Wave_Ht         25
Wave_Prd        28
IntChl        1710
Dry_T          220
dtype: int64

In [32]:
data.to_csv("data_merge.csv")

In [33]:
data_part = data[400000:600000]


In [35]:
data_part.to_csv("data_merge_part.csv")